In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [2]:
!pip install pyomo
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 110.8 MB/s eta 0:00:00


In [3]:
!apt-get install -y glpk-utils # does not attain a global optimum
!apt-get install -y coinor-cbc # chosen solver for MILP

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 35 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsuitesparseconfig5 amd64 1:5.10.1+dfsg-4build1 [10.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libamd2 amd64 1:5.10.1+dfsg-4build1 [21.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcolamd2 amd64 1:5.10.1+dfsg-4build1 [18.0 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libglpk40 amd64 5.0-1 [361 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 glpk-ut

In [4]:
!conda install -c conda-forge ipopt -y

Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - ipopt


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ampl-asl-1.0.0             |       h5888daf_2         504 KB  conda-forge
    ca-certificates-2025.7.14  |       hbd8a1cb_0         152 KB  conda-forge
    certifi-2025.7.14          |     pyhd8ed1ab_0         156 KB  conda-forge
    conda-24.11.3              |  py311h38be061_0         1.1 MB  conda-forge
    ipopt-3.14.17              |       h7fd866c_2         984 KB  conda-forge
    libblas-3.9.0              |31_h59b9bed_openblas          16 KB  conda-forge
    libcblas-3.9.0             |31_he106b2a_openblas          16 KB  conda-forge
    libgfortran-14.2.0         |       h69a702a_2          52 KB  conda-forge
    libgfortran-ng-14.2.0      |       h69a702a_

In [5]:
!which ipopt

/usr/local/bin/ipopt


In [6]:
from pyomo.environ import *
import math
import numpy as np
from pyomo.core import Constraint
import random
import gurobipy
import copy
import pandas as pd
import pickle
import time

In [7]:
def build_model(N, params):
    m = ConcreteModel()

    # 1) Index set
    m.N = Set(initialize=N, ordered=True)

    # 2) Parameters (constants)
    m.Phi_SE  = Param(initialize=params['Phi_SE'], mutable=False)
    m.Phi_EE  = Param(initialize=params['Phi_EE'], mutable=False)
    m.zeta    = Param(initialize=params['zeta'],   mutable=False)
    m.pr      = Param(initialize=params['pr'],     mutable=False)
    m.eps     = Param(initialize=params['eps'],    mutable=False)
    m.Cn      = Param(m.N, initialize=params['Cn'], mutable=False)
    m.B       = Param(initialize=params['B'],      mutable=False)
    m.sigma2  = Param(initialize=params['sigma2'], mutable=False)

    m.Rth     = Param(m.N, initialize=params['Rth'], mutable=False)
    m.Pth     = Param(m.N, initialize=params['Pth'], mutable=False)
    m.fmin    = Param(m.N, initialize=params['fmin'], mutable=False)
    m.fmax    = Param(m.N, initialize=params['fmax'], mutable=False)

    m.g2      = Param(m.N, initialize=params['g2'], mutable=False)
    m.order   = Param(m.N, initialize=params['order'], mutable=False)

    # This is the SCA linearization point, updated each iter:
    m.xk      = Param(m.N, initialize={n: 0.0 for n in N},
                      mutable=True)

    # 3) Variables
    m.f   = Var(m.N, bounds=lambda mod,n: (mod.fmin[n], mod.fmax[n]))
    m.rho = Var(m.N, domain=Reals)   # log(p_n)
    m.a   = Var(m.N, domain=Reals)   # aux. rate
    m.x   = Var(m.N, domain=Reals)   # new SCA slack

    # 4) Objective
    def obj_rule(mod):
        return mod.Phi_SE * sum(mod.a[n] for n in mod.N) \
             - mod.Phi_EE * sum(mod.zeta*exp(mod.rho[n])
                                + mod.pr
                                + mod.eps*mod.f[n]**3
                                for n in mod.N)
    m.obj = Objective(rule=obj_rule, sense=1)  # maximize

    # 5) Constraints
    def c_rate_min(mod,n):
        return mod.a[n] >= log(mod.Rth[n])
    m.c_rate_min = Constraint(m.N, rule=c_rate_min)

    def c_power_cap(mod,n):
        return mod.zeta*exp(mod.rho[n]) + mod.pr + mod.eps*mod.f[n]**3 \
               <= mod.Pth[n]
    m.c_power_cap = Constraint(m.N, rule=c_power_cap)

    def c_sca_linear(mod,n):
        # first‐order Taylor of exp(a)-f/Cn at xk:
        return exp(mod.a[n]) - mod.f[n]/mod.Cn[n] \
             <= exp(mod.xk[n]) + exp(mod.xk[n])*(mod.x[n] - mod.xk[n])
    m.c_sca_linear = Constraint(m.N, rule=c_sca_linear)

    def c_rate_sca(mod,n):
        # build NOMA‐SIC denominator
        denom = sum(exp(mod.rho[i])*mod.g2[i]
                    for i in mod.N
                    if mod.order[i] < mod.order[n]) \
                + mod.sigma2
        sinr = exp(mod.rho[n])*mod.g2[n]/denom
        # log2 => ln/ln2
        return exp(mod.x[n]) <= mod.B*log(1+sinr)/log(2)
    m.c_rate_sca = Constraint(m.N, rule=c_rate_sca)

    return m

In [8]:
if __name__=='__main__':
    # --- 1) Artificial Problem Data  ---
    N = [1,2,3]   # example 3 users
    params = {
        'Phi_SE': 1.0, 'Phi_EE': 0.5,
        'zeta':1e-3, 'pr':0.1, 'eps':1e-27,
        'Cn': {1:1e8,2:1e8,3:1e8},
        'B': 1e6, 'sigma2':1e-9,
        'Rth':{1:1,2:1,3:1},
        'Pth':{1:0.5,2:0.5,3:0.5},
        'fmin':{1:0.5e9,2:0.5e9,3:0.5e9},
        'fmax':{1:2e9,2:2e9,3:2e9},
        'g2':{1:1.0,2:0.8,3:0.5},
        'order':{1:1,2:2,3:3},
    }

    # --- 2) Model Building ---
    model = build_model(N, params)
    solver = SolverFactory('ipopt')
    solver.options['tol'] = 1e-7

    # --- 3) SCA Initialization ---
    xk = {n: 0.0 for n in N}
    max_iter = 20
    tol = 1e-4

    for it in range(1, max_iter+1):
        # 3a) Updating Linearization Point Param
        for n in N:
            model.xk[n] = xk[n]

        # 3b) Solving the Convex Subproblem
        result = solver.solve(model, tee=False)

        # 3c) Extracting New x and Objective
        x_new = {n: value(model.x[n]) for n in N}
        obj_val = value(model.obj)

        # 3d) Checking Convergence
        diff = max(abs(x_new[n] - xk[n]) for n in N)
        print(f"Iter {it}: obj = {obj_val:.6f}, ||Δx||_∞ = {diff:.6e}")
        if diff < tol:
            print("Converged!")
            break

        # 3e) Update Variables and Parameters for Next Round
        xk = x_new

    # --- 4) Final Solution ---
    f_opt   = {n: value(model.f[n])   for n in N}
    rho_opt = {n: value(model.rho[n]) for n in N}
    a_opt   = {n: value(model.a[n])   for n in N}
    x_opt   = xk

    print("f*:   ", f_opt)
    print("rho*:", rho_opt)
    print("a*:   ", a_opt)
    print("x*:   ", x_opt)


Iter 1: obj = -0.750000, ||Δx||_∞ = 1.321330e+01
Iter 2: obj = -0.750000, ||Δx||_∞ = 1.746212e+00
Iter 3: obj = -0.746690, ||Δx||_∞ = 4.182028e-01
Iter 4: obj = -0.746571, ||Δx||_∞ = 1.223282e-01
Iter 5: obj = -0.746575, ||Δx||_∞ = 3.719172e-02
Iter 6: obj = -0.746576, ||Δx||_∞ = 1.151936e-02
Iter 7: obj = -0.746577, ||Δx||_∞ = 3.609125e-03
Iter 8: obj = -0.746577, ||Δx||_∞ = 1.142947e-03
Iter 9: obj = -0.746577, ||Δx||_∞ = 3.665525e-04
Iter 10: obj = -0.746577, ||Δx||_∞ = 1.194394e-04
Iter 11: obj = -0.746577, ||Δx||_∞ = 3.970320e-05
Converged!
f*:    {1: 734698094.9910939, 2: 734694452.3772076, 3: 721384129.6434835}
rho*: {1: -12.977443826530013, 2: -5.007682844046605, 3: 3.2025534273780796}
a*:    {1: -9.09096909216206e-10, 2: -9.090969092102768e-10, 3: -9.090969092087945e-10}
x*:    {1: 15.541185465778952, 2: 15.541246698950815, 3: 15.540212243952952}
